# Cryptol slicer notebook

This notebook calls the Haskell `toy-cryptol-ast` executable to slice Cryptol modules
into per-definition files. It assumes:

1. You have the `toy-cryptol-ast` repo checked out.
2. You can run `cabal build` or `cabal run toy-cryptol-ast` successfully from that repo.
3. You have a local checkout of `cryptol-specs` (or any directory tree of `.cry` files).


In [1]:
from pathlib import Path
import subprocess

# --- CONFIG: EDIT THESE FOR YOUR MACHINE ---

# Root of your repo/workspace that contains both the Cryptol specs and the file list
BASE_REPO = Path("/Users/josh/SecurityAnalytics")   # <-- adjust if needed

# Path to the file listing all .cry files (e.g., 0_long_files.txt)
FILE_LIST = BASE_REPO / "0_long_files.txt"          # <-- adjust if needed

# Root directory where all slice directories will be created
OUTPUT_ROOT = BASE_REPO / "cryptol_slices"

# Directory containing the cabal project for toy-cryptol-ast
CABAL_PROJECT_ROOT = BASE_REPO / "toy-cryptol-ast"  # directory with toy-cryptol-ast.cabal


In [2]:
def run_haskell_slicer(cry_path: Path, out_dir: Path):
    """
    Call: cabal run toy-cryptol-ast -- <file> <OUTDIR>
    from within CABAL_PROJECT_ROOT.
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    cmd = [
        "cabal",
        "run",
        "toy-cryptol-ast",
        "--",
        str(cry_path),
        str(out_dir),
    ]

    print("Running:", " ".join(cmd))
    result = subprocess.run(
        cmd,
        cwd=str(CABAL_PROJECT_ROOT),  # run inside the cabal project
        capture_output=True,
        text=True,
    )

    if result.returncode != 0:
        print(f"❌ Error ({result.returncode}) on {cry_path}")
        if result.stdout.strip():
            print("STDOUT:\n", result.stdout)
        if result.stderr.strip():
            print("STDERR:\n", result.stderr)
    else:
        print(f"✅ OK: {cry_path}")
        if result.stdout.strip():
            print("STDOUT:\n", result.stdout)


In [3]:
def iter_file_list(file_list: Path):
    """
    Yield each non-empty, non-comment line from FILE_LIST.
    Lines should be repo-relative paths like:
      cryptol-specs/Primitive/Symmetric/Cipher/Block/KATAN.cry
    """
    with file_list.open() as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue
            if line.startswith("#"):
                continue
            yield line


def cry_rel_to_paths(rel: str):
    """
    Given a relative .cry path from FILE_LIST, return:
      (absolute_input_path, absolute_output_dir_path)
    Output dir is:
      OUTPUT_ROOT / relative_path_without_.cry_suffix
    """
    cry_rel = Path(rel)
    cry_abs = BASE_REPO / cry_rel

    # Example:
    #   rel: cryptol-specs/Primitive/Symmetric/Cipher/Block/KATAN.cry
    #   out: OUTPUT_ROOT/cryptol-specs/Primitive/Symmetric/Cipher/Block/KATAN/
    out_dir = OUTPUT_ROOT / cry_rel.with_suffix("")

    return cry_abs, out_dir


print(f"Reading list from: {FILE_LIST}")
print(f"Output root:       {OUTPUT_ROOT}")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

for rel in iter_file_list(FILE_LIST):
    cry_path, out_dir = cry_rel_to_paths(rel)

    if not cry_path.is_file():
        print(f"⚠️  Skipping {cry_path} (file not found)")
        continue

    print("\n=== Slicing:", cry_path, "===")
    print("  → output dir:", out_dir)
    run_haskell_slicer(cry_path, out_dir)


Reading list from: /Users/josh/SecurityAnalytics/0_long_files.txt
Output root:       /Users/josh/SecurityAnalytics/cryptol_slices

=== Slicing: /Users/josh/SecurityAnalytics/AES-GCM-SIV-proof/proof/asm/deps/saw-script/examples/openssl_aes/AES128TBox.cry ===
  → output dir: /Users/josh/SecurityAnalytics/cryptol_slices/AES-GCM-SIV-proof/proof/asm/deps/saw-script/examples/openssl_aes/AES128TBox
Running: cabal run toy-cryptol-ast -- /Users/josh/SecurityAnalytics/AES-GCM-SIV-proof/proof/asm/deps/saw-script/examples/openssl_aes/AES128TBox.cry /Users/josh/SecurityAnalytics/cryptol_slices/AES-GCM-SIV-proof/proof/asm/deps/saw-script/examples/openssl_aes/AES128TBox
✅ OK: /Users/josh/SecurityAnalytics/AES-GCM-SIV-proof/proof/asm/deps/saw-script/examples/openssl_aes/AES128TBox.cry
STDOUT:
 Parsed 1 module(s).


=== Slicing: /Users/josh/SecurityAnalytics/aws-lc-verification/cryptol-specs/Primitive/Symmetric/Cipher/Stream/chacha20.cry ===
  → output dir: /Users/josh/SecurityAnalytics/cryptol_slices/